In [15]:
import socket
import hashlib
import json
import time
import matplotlib.pyplot as plt
import os
import csv

In [13]:
HOST = '127.0.0.1'    
PORT = 13854
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))

In [16]:
# # Authenticate
# client_phrase =b"I am a woman with great aspirations"
# client_key = hashlib.sha1(client_phrase).hexdigest()

# app_name = "Suchi EEG"

# auth_dict = {"appName":app_name,"appKey":client_key}

# auth_msg = json.dumps(auth_dict).encode('ascii')
# s.send(auth_msg)

In [36]:
#set a server config and validate
server_config = json.dumps({"enableRawOutput": True, "format": "Json"}).encode('ascii')
s.send(server_config)

43

In [54]:
def get_data(secs=30):
    event=input('Enter \n1 for Baseline \n2 for Game\n or 0 for other \n')
    
    
    if int(event)==2:
            event_name='Game_'
    elif int(event)==1:
            event_name='Baseline_'
    else:
        event_name='Other_'
    
    if not(os.path.exists('data')):
        print('Creating folder: data..')
        os.makedirs('data')
    timet=time.strftime("%d-%m_%H-%M-%S")
    logdir =event_name+timet +'.json'
    logdir = os.path.join('data', logdir)
    feeling=input('How are you feeling? ')
    task=input('Is there any specific task name? ')
    secs=input('Input recording time: ')
 
    
    with open(logdir, 'w', encoding="ascii", newline='\r\n') as outfile:
        ref = time.time()    
        while (time.time() - ref) < float(secs):
            response = s.recv(2048).decode('ascii')
            print((time.time() - ref))
        
            outfile.write(response)
            print(">>>", response)
            
        metadata={"metadata":{"Feeling":feeling,"Time":timet,"Task":task}}
        outfile.write(json.dumps(metadata))

    return logdir



In [55]:


def plot_raw_from_file(fpath):
    print(fpath)
    eegs = []
    for line in open(fpath,'r'):
        line=line.strip()
        if line!="":
            raw_eeg = json.loads(line)
            if 'rawEeg' in raw_eeg:
                eegs.append(raw_eeg['rawEeg'])
#             if 'metadata' in raw_eeg:
#                 print(raw_eeg)
                
    fig = plt.figure(figsize=(15,4))
    plt.plot(eegs,'r')
    plt.title(fpath)
    plt.show()
#     return eegs

In [56]:

file=get_data()
file



Enter 
1 for Baseline 
2 for Game
 or 0 for other 
1
How are you feeling? 1
Is there any specific task name? 1
Input recording time: 1
10.437114953994751
>>> {"poorSignalLevel":-1}


'data/Baseline_22-11_10-52-37.json'

In [20]:
plot_raw_from_file(file)

NameError: name 'plot_raw_from_file' is not defined

In [18]:
import pandas as pd

path='data'
files=!ls data
# files
df=pd.DataFrame(columns=['eeg','activity','date','time','metadata'])
for file in files:
    print(file)
    if file.endswith('.json'):
        eegs = []
        filename=path+'/'+file
        for line in open(filename,'r'):
            line=line.strip()
            if line!="":
                try:
                    raw_eeg = json.loads(line)
                    if 'rawEeg' in raw_eeg:
                        eegs.append(raw_eeg['rawEeg'])
                    if 'metadata' in raw_eeg:
                        metadata=raw_eeg
                except:
                    print('\n>>>ERROR: cant read a line in',file)
                    pass
        filesplit=file.split('_')
        
        activity=filesplit[0]
        date=filesplit[1]
#         time=filesplit[2][:-5]
    #     print(activity,date,time,metadata)
        df2=pd.DataFrame([[eegs,activity,date,time,metadata]],columns=['eeg','activity','date','time','metadata'])
        df=pd.concat([df2,df])      
    


Baseline_02-11_22-02-54.json
Baseline_06-11_00-42-47.json

>>>ERROR: cant read a line in Baseline_06-11_00-42-47.json
Baseline_07-11_19-52-03.json
Baseline_10-11_05-12-38.json
Baseline_12-11_15-27-43.json
Baseline_22-11_10-38-54.json

>>>ERROR: cant read a line in Baseline_22-11_10-38-54.json
Baseline_22-11_10-48-19.json

>>>ERROR: cant read a line in Baseline_22-11_10-48-19.json
Baseline_22-11_10-50-44.json
Baseline_22-11_10-52-37.json

>>>ERROR: cant read a line in Baseline_22-11_10-52-37.json
Baseline_30-10_10-05-54.json
Baseline_31-10_09-03-29.json
Game_02-11_22-08-30.json
Game_06-11_00-51-30.json
Game_07-11_19-54-37.json
Game_10-11_05-18-22.json
Game_12-11_15-30-12.json
Game_12-11_15-32-28.json
Game_30-10_10-13-17.json
Game_31-10_09-10-31.json
Other_22-11_10-44-12.json

>>>ERROR: cant read a line in Other_22-11_10-44-12.json

>>>ERROR: cant read a line in Other_22-11_10-44-12.json
Other_30-10_10-19-28.json


In [ ]:
df.shape

In [19]:
df.to_csv('data_after_29_oct.csv',index=False)